In [1]:
%cd ~/src/second-best-bench
from src.plots.common import *

/home/yimingz3/src/second-best-bench


In [2]:
def compute_summary(df):
    
    summary = df.groupby(['model_family', 'model_alias']).agg(
        mean_distinct=('distinct', 'mean'),
        mean_utility=('utility', 'mean')
    ).reset_index()
    
    summary['distinct'] = summary['mean_distinct'] + 1
    
    return summary[['model_family', 'model_alias', 'distinct', 'mean_utility']]

eval_data = compute_summary(model_scores)
eval_data = eval_data.rename(columns={'mean_utility': 'utility'})


In [3]:
def add_dummy_separators(df):
    # Get unique model families in the desired order
    families = sorted(eval_data["model_family"].unique())
    
    # Create a new dataframe with dummy rows
    new_rows = []
    for i, family in enumerate(families):
        if family != "OpenAI":  # Add separator before all families except the first one
            # Create a dummy row with null values
            dummy = {'model_family': family, 'model_alias': " " * (i+1), 
                     'distinct': None, 'utility': None}
            new_rows.append(dummy)
        
        # Add all rows for this family
        family_rows = df[df['model_family'] == family].to_dict('records')
        new_rows.extend(family_rows)
    
    return pd.DataFrame(new_rows)

# Apply the function to create separated data
separated_data = add_dummy_separators(eval_data)

# Create utility chart without text labels
utility_chart = alt.Chart(separated_data).mark_bar().encode(
    x=alt.X('model_alias:N', title='', sort=model_order, axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('utility:Q', title='Utility'),
    color=alt.Color('model_family:N', title='Model family', 
                   scale=alt.Scale(domain=list(eval_data['model_family'].unique())))
).properties(
    width=600,
    height=200
)

# Add text labels on top of bars
utility_chart = utility_chart + utility_chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-5
).encode(
    text=alt.Text('utility:Q', format='.2f')
)


# Save and display the chart without text labels
utility_chart.save("plots/utility.json")
utility_chart


alt.LayerChart(...)

In [4]:
distinct_chart = alt.Chart(separated_data).mark_bar().encode(
    x=alt.X('model_alias:N', title='', sort=model_order, axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('distinct:Q', title='Distinct Generations (out of 10)'),
    color=alt.Color('model_family:N', title='Model family', 
                   scale=alt.Scale(domain=list(eval_data['model_family'].unique()))),
    tooltip=['model_alias', 'model_family', 'distinct']
).properties(
    width=600,
    height=200
)

# Add text labels on top of bars
distinct_text = distinct_chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-5
).encode(
    text=alt.Text('distinct:Q', format='.2f')
)

# Combine chart and text
final_distinct_chart = distinct_chart + distinct_text

final_distinct_chart.save("plots/distinct.json")
# Display the chart
final_distinct_chart

alt.LayerChart(...)